In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist         # библиотека базы выборок Mnist
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [2]:
data=pd.read_csv('movies.csv')

In [3]:
data=data.dropna()#Убираем пропущенные значения

In [4]:
data.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')

# Предобработка 

In [5]:
data['reviews_from_users']

488      471.0
1005     270.0
2336     262.0
2687     229.0
3100     808.0
         ...  
60090      6.0
60112     13.0
60117     22.0
60118    104.0
60145    321.0
Name: reviews_from_users, Length: 5246, dtype: float64

'imdb_title_id', -удаляем так как не несет смысла+ 

'title',--удаляем так как не несет смысла +

'original_title',-удаляем так как почти всегда совпадает с title+

'year',-без изменений+

'date_published',-удаляем+

 'genre',-используем векторайзер+
       
'duration',-оставляем без изменений+
       
'country',-используем дамми переменные+
       
'language',используем векторайзер+
       
'director',-дамми редкие на other+
       
'writer',-дамми редкие на other+
       
'production_company',-дамми редкие на other+
       
'actors',-используем векторайзер+
       
'description',-удаляем так как усложняют модель+
       
'avg_vote',-удаляем так как таких данных у нас не будет+
       
'votes',-удаляем так как таких данных у нас не будет+
       
'budget',-убрать все что не в $+
       
'usa_gross_income',-учесть инфляцию либо убираем(убираем)+
       
 'worlwide_gross_income',-учесть инфляцию либо убираем(убираем)+
       
'metascore',-без изменений целевая переменная+
       
'reviews_from_users',-удаляем так как таких данных у нас не будет+

'reviews_from_critics'],-удаляем так как таких данных у нас не будет+
       

Удаляем лишнее

In [6]:
data=data.drop(['imdb_title_id','date_published','title','original_title','usa_gross_income','worlwide_gross_income','description',"avg_vote","votes",'reviews_from_users','reviews_from_critics'],axis=1)

In [7]:
data

,year,genre,duration,country,language,director,writer,production_company,actors,budget,metascore
488,1927,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",DEM 6000000,98.0
1005,1931,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, ...",$ 1500000,99.0
2336,1936,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergm...",$ 1500000,96.0
2687,1937,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti...",$ 1499000,95.0
3100,1939,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh,...",$ 3977000,97.0
...,...,...,...,...,...,...,...,...,...,...,...
60090,2013,Comedy,85,"Spain, Ireland, France","Catalan, English, Spanish, Japanese",Roger Gual,"Javier Calvo, Silvia González Laá",Subotica Entertainment,"Jan Cornet, Claudia Bassols, Vicenta N'Dongo, ...",EUR 2700000,29.0
60112,2013,"Drama, Musical, Romance",110,"South Korea, USA",English,Duane Adler,Duane Adler,CJ Entertainment,"Derek Hough, BoA, Will Yun Lee, Wesley Jonatha...",$ 15000000,40.0
60117,2011,"Comedy, Drama, Romance",108,France,"French, Swedish","David Foenkinos, Stéphane Foenkinos","David Foenkinos, David Foenkinos",2.4.7. Films,"Audrey Tautou, François Damiens, Bruno Todesch...",EUR 7000000,53.0
60118,2012,"Drama, Mystery, Thriller",102,"France, Germany","English, German",Brian De Palma,"Brian De Palma, Natalie Carter",SBS Productions,"Rachel McAdams, Noomi Rapace, Karoline Herfurt...",$ 20000000,53.0


Отбрасываем фильмы до 2000

In [8]:
data = data[data.year > 2008]


In [9]:
new_data=[]
for budget in data["budget"]:
    if '$' not in budget:
        budget=''
    new_data.append(budget)
data["budget"]=new_data
data = data[data.budget != '']

C:\Users\sevas\AppData\Local\Temp\ipykernel_17656\3231305122.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["budget"]=new_data


In [10]:
new_data=[]
for budget in data["budget"]:
    if '$' not in budget:
        budget=''
    new_data.append(budget)
data["budget"]=new_data
data = data[data.budget != '']

In [11]:
data["budget"] = data["budget"].replace('[\$]', '', regex=True).astype(float)
#data["budget"] = data["budget"].replace('[GBP,]', '', regex=True)#.astype(float)
#data["budget"] = data["budget"].replace('[EUR,]', '', regex=True)

In [12]:
data

,year,genre,duration,country,language,director,writer,production_company,actors,budget,metascore
40953,2010,"Drama, Mystery, Thriller",104,USA,English,Allen Wolf,Allen Wolf,Morning Star Pictures,"Philip Winchester, Tim Draxl, Lacey Chabert, A...",1000000.0,33.0
40988,2009,"Animation, Drama, Fantasy",100,USA,"English, Russian",Henry Selick,"Henry Selick, Neil Gaiman",Focus Features,"Dakota Fanning, Teri Hatcher, Jennifer Saunder...",60000000.0,80.0
41298,2012,"Adventure, Drama, Romance",124,"France, USA, UK, Mexico, Canada, Brazil, Germa...","English, French",Walter Salles,"Jack Kerouac, Jose Rivera",MK2 Productions,"Sam Riley, Garrett Hedlund, Kristen Stewart, A...",25000000.0,56.0
42089,2013,"Comedy, Drama, Fantasy",114,"USA, UK","English, Spanish, Icelandic",Ben Stiller,"Steve Conrad, Steve Conrad",Twentieth Century Fox,"Ben Stiller, Kristen Wiig, Jon Daly, Kathryn H...",90000000.0,54.0
42141,2009,"Adventure, Drama, War",153,"Germany, USA","English, German, French, Italian",Quentin Tarantino,Quentin Tarantino,Universal Pictures,"Brad Pitt, Mélanie Laurent, Christoph Waltz, E...",70000000.0,69.0
...,...,...,...,...,...,...,...,...,...,...,...
60069,2013,Comedy,81,USA,English,Michelle Schumacher,"Randle Schumacher, Eric Radzan",Rubber Tree Productions,"J.K. Simmons, Tim Allen, Scott Caan, Basil Hof...",1000000.0,9.0
60079,2014,"Comedy, Romance",100,USA,English,Steve Pink,"Leslye Headland, Tim Kazurinsky",Screen Gems,"Kevin Hart, Michael Ealy, Regina Hall, Joy Bry...",12500000.0,62.0
60112,2013,"Drama, Musical, Romance",110,"South Korea, USA",English,Duane Adler,Duane Adler,CJ Entertainment,"Derek Hough, BoA, Will Yun Lee, Wesley Jonatha...",15000000.0,40.0
60118,2012,"Drama, Mystery, Thriller",102,"France, Germany","English, German",Brian De Palma,"Brian De Palma, Natalie Carter",SBS Productions,"Rachel McAdams, Noomi Rapace, Karoline Herfurt...",20000000.0,53.0


Заменим малоизвестных режиссеров,сценаристов,страны,компании на 'other'

In [13]:
vc = data['director'].value_counts()
data.loc[data['director'].isin(vc[vc < 10].index), 'director'] = 'other'

In [14]:
vc = data['writer'].value_counts()
data.loc[data['writer'].isin(vc[vc < 5].index), 'writer'] = 'other'

In [15]:
vc = data['country'].value_counts()
data.loc[data['country'].isin(vc[vc < 10].index), 'country'] = 'other'

In [16]:
data.loc[data["country"] == "Germany, USA", "country"] = "Germany, USA"
data.loc[data["country"] == "USA, Germany", "country"] = "Germany, USA"

In [17]:
vc = data['production_company'].value_counts()
data.loc[data['production_company'].isin(vc[vc < 15].index), 'production_company'] = 'other'

Форматируем дату

In [18]:
#data['date_published'] = pd.to_datetime(data['date_published'], errors='coerce')

In [19]:
#data['date_published']=data['date_published'].dt.strftime("%Y%m%d").astype(int)

Применяем векторайзер для жанра

In [20]:
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(data["genre"].values.astype('U')) 
vectorized_data=pd.DataFrame(X.todense(),columns=vectorizer.get_feature_names_out())#в случае появления ошибок убрать _out
pd.concat([data, vectorized_data], axis=1).reindex(vectorized_data.index)
data=data.drop(["genre"],axis=1)

In [21]:
vectorized_data

,action,adventure,animation,biography,comedy,crime,drama,family,fantasy,fi,...,horror,music,musical,mystery,romance,sci,sport,thriller,war,western
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,0,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1142,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1143,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1144,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0


Аналогично для языков

In [22]:
data

,year,duration,country,language,director,writer,production_company,actors,budget,metascore
40953,2010,104,USA,English,other,other,other,"Philip Winchester, Tim Draxl, Lacey Chabert, A...",1000000.0,33.0
40988,2009,100,USA,"English, Russian",other,other,Focus Features,"Dakota Fanning, Teri Hatcher, Jennifer Saunder...",60000000.0,80.0
41298,2012,124,other,"English, French",other,other,other,"Sam Riley, Garrett Hedlund, Kristen Stewart, A...",25000000.0,56.0
42089,2013,114,"USA, UK","English, Spanish, Icelandic",other,other,Twentieth Century Fox,"Ben Stiller, Kristen Wiig, Jon Daly, Kathryn H...",90000000.0,54.0
42141,2009,153,other,"English, German, French, Italian",other,other,Universal Pictures,"Brad Pitt, Mélanie Laurent, Christoph Waltz, E...",70000000.0,69.0
...,...,...,...,...,...,...,...,...,...,...
60069,2013,81,USA,English,other,other,other,"J.K. Simmons, Tim Allen, Scott Caan, Basil Hof...",1000000.0,9.0
60079,2014,100,USA,English,other,other,Screen Gems,"Kevin Hart, Michael Ealy, Regina Hall, Joy Bry...",12500000.0,62.0
60112,2013,110,other,English,other,other,other,"Derek Hough, BoA, Will Yun Lee, Wesley Jonatha...",15000000.0,40.0
60118,2012,102,other,"English, German",other,other,other,"Rachel McAdams, Noomi Rapace, Karoline Herfurt...",20000000.0,53.0


In [23]:
data["language"]

40953                             English
40988                    English, Russian
41298                     English, French
42089         English, Spanish, Icelandic
42141    English, German, French, Italian
                       ...               
60069                             English
60079                             English
60112                             English
60118                     English, German
60145                             Persian
Name: language, Length: 1146, dtype: object

In [24]:
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(data["language"].values.astype('U')) 
vectorized_data=pd.DataFrame(X.todense(),columns=vectorizer.get_feature_names_out())#в случае появления ошибок убрать _out
#pd.concat([data, vectorized_data], axis=1).reindex(vectorized_data.index)
data=data.drop(["language"],axis=1)

Аналогично для актеров

In [25]:
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(data["actors"].values.astype('U')) 
vectorized_data=pd.DataFrame(X.todense(),columns=vectorizer.get_feature_names_out())#в случае появления ошибок убрать _out
#pd.concat([data, vectorized_data], axis=1).reindex(vectorized_data.index)
data=data.drop(["actors"],axis=1)

Создадим Dummy для стран

In [26]:
data=pd.get_dummies(data,columns=["country","director","writer","production_company"])

In [27]:
data.columns

Index(['year', 'duration', 'budget', 'metascore', 'country_France',
       'country_UK', 'country_UK, USA', 'country_USA', 'country_USA, Canada',
       'country_USA, UK', 'country_other', 'director_other',
       'writer_Joel Coen, Ethan Coen', 'writer_Jon Lucas, Scott Moore',
       'writer_Patrick Melton, Marcus Dunstan', 'writer_other',
       'production_company_Columbia Pictures', 'production_company_DreamWorks',
       'production_company_Focus Features',
       'production_company_Fox 2000 Pictures', 'production_company_Lionsgate',
       'production_company_New Line Cinema',
       'production_company_Paramount Pictures',
       'production_company_Screen Gems',
       'production_company_Summit Entertainment',
       'production_company_Twentieth Century Fox',
       'production_company_Universal Pictures',
       'production_company_Walt Disney Pictures',
       'production_company_Warner Bros.', 'production_company_other'],
      dtype='object')

Подготовим тестовую и обучающие выборки

In [35]:
y=data["metascore"].values
X=data.drop(["metascore"],axis=1).values
#биас
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])

In [36]:
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)

In [37]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')

In [38]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)


Выберем и обучим модель

In [39]:
clf = RandomForestRegressor(max_depth=40,n_estimators=70)
clf.fit(X_train,y_train)


RandomForestRegressor(max_depth=40, n_estimators=70)

In [40]:
y_pred=clf.predict(X_valid)

In [41]:
print_regression_metrics(y_valid, y_pred)

MSE = 264.68, RMSE = 16.27
